In [ ]:

"""
SCRIPT COMPLETO: Extracción de Datos Agrícolas para Videojuego
Región: Cusco, Perú
Datos: NDVI, Temperatura, Precipitación
Autor: Para Hackathon FarmGuardians
"""

# ============================================
# PASO 1: INSTALACIÓN Y CONFIGURACIÓN
# ============================================
# Ejecutar en Colab:
# !pip install earthengine-api geemap pandas numpy matplotlib

import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ============================================
# PASO 2: AUTENTICACIÓN
# ============================================
# Primera vez: ee.Authenticate()
ee.Initialize(project='tu-proyecto-aqui')  # Cambiar por tu proyecto

# ============================================
# PASO 3: DEFINIR ÁREA DE CUSCO
# ============================================

# Opción A: Punto específico (Plaza de Armas, Cusco)
cusco_point = ee.Geometry.Point([-71.9675, -13.5319])

# Opción B: Región completa de Cusco
cusco_region = ee.Geometry.Polygon([
    [-72.5, -13.0],
    [-71.0, -13.0],
    [-71.0, -14.5],
    [-72.5, -14.5],
    [-72.5, -13.0]
])

# Opción C: Valle Sagrado (zona agrícola importante)
valle_sagrado = ee.Geometry.Polygon([
    [-72.2, -13.2],
    [-71.8, -13.2],
    [-71.8, -13.4],
    [-72.2, -13.4],
    [-72.2, -13.2]
])

# Elegir geometría para análisis
area_estudio = cusco_point  # Cambiar según necesites

# ============================================
# PASO 4: FUNCIONES DE EXTRACCIÓN
# ============================================

def extraer_ndvi_sentinel2(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de Sentinel-2 (10m resolución)
    Ideal para parcelas específicas
    """
    def calcular_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return ndvi.copyProperties(image, ['system:time_start'])

    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate(fecha_inicio, fecha_fin) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
        .map(calcular_ndvi)

    return collection


def extraer_ndvi_modis(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de MODIS (250m resolución)
    Mejor cobertura temporal, datos desde 2000
    """
    collection = ee.ImageCollection('MODIS/061/MOD13Q1') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('NDVI')

    def escalar_ndvi(image):
        return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

    return collection.map(escalar_ndvi)


def extraer_temperatura(geometry, fecha_inicio, fecha_fin):
    """
    Extrae temperatura superficial (°C)
    """
    collection = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('LST_Day_1km')

    def convertir_celsius(image):
        celsius = image.multiply(0.02).subtract(273.15)
        return celsius.copyProperties(image, ['system:time_start'])

    return collection.map(convertir_celsius)


def extraer_precipitacion(geometry, fecha_inicio, fecha_fin):
    """
    Extrae precipitación diaria (mm)
    """
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('precipitation')

    return collection


def collection_a_dataframe(image_collection, geometry, escala, nombre_banda):
    """
    Convierte ImageCollection a DataFrame de pandas
    """
    def extraer_valor(image):
        valor = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=escala,
            maxPixels=1e9
        ).get(nombre_banda)

        fecha = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

        return ee.Feature(None, {
            'fecha': fecha,
            nombre_banda: valor
        })

    features = image_collection.map(extraer_valor)
    info = features.getInfo()

    datos = [f['properties'] for f in info['features']]
    df = pd.DataFrame(datos)
    df['fecha'] = pd.to_datetime(df['fecha'])

    return df


# ============================================
# PASO 5: EXTRAER DATOS (2023-2024)
# ============================================

print("🌱 Extrayendo datos de Cusco para videojuego...")

fecha_inicio = '2023-01-01'
fecha_fin = '2024-12-31'

# NDVI (usa Sentinel-2 para alta resolución)
print("📊 Extrayendo NDVI...")
ndvi_collection = extraer_ndvi_sentinel2(area_estudio, fecha_inicio, fecha_fin)
ndvi_df = collection_a_dataframe(ndvi_collection, area_estudio, 10, 'NDVI')

# Temperatura
print("🌡️ Extrayendo temperatura...")
temp_collection = extraer_temperatura(area_estudio, fecha_inicio, fecha_fin)
temp_df = collection_a_dataframe(temp_collection, area_estudio, 1000, 'LST_Day_1km')
temp_df.rename(columns={'LST_Day_1km': 'temperatura_celsius'}, inplace=True)

# Precipitación
print("💧 Extrayendo precipitación...")
precip_collection = extraer_precipitacion(area_estudio, fecha_inicio, fecha_fin)
precip_df = collection_a_dataframe(precip_collection, area_estudio, 5000, 'precipitation')
precip_df.rename(columns={'precipitation': 'precipitacion_mm'}, inplace=True)

print("✅ Datos extraídos exitosamente!")

# ============================================
# PASO 6: COMBINAR Y LIMPIAR DATOS
# ============================================

# Unir todos los dataframes
datos_completos = ndvi_df.merge(temp_df, on='fecha', how='outer') \
                         .merge(precip_df, on='fecha', how='outer')

# Ordenar por fecha
datos_completos = datos_completos.sort_values('fecha').reset_index(drop=True)

# Interpolar valores faltantes
datos_completos['NDVI'] = datos_completos['NDVI'].interpolate(method='linear')
datos_completos['temperatura_celsius'] = datos_completos['temperatura_celsius'].interpolate(method='linear')
datos_completos['precipitacion_mm'] = datos_completos['precipitacion_mm'].fillna(0)

# ============================================
# PASO 7: CREAR MÉTRICAS PARA EL JUEGO
# ============================================

def calcular_metricas_juego(df):
    """
    Crea métricas útiles para mecánicas de videojuego
    """
    # Salud del cultivo (0-100)
    df['salud_cultivo'] = (df['NDVI'] * 100).clip(0, 100)

    # Estrés por calor
    df['estres_calor'] = pd.cut(
        df['temperatura_celsius'],
        bins=[-np.inf, 15, 25, 35, np.inf],
        labels=['Frío', 'Óptimo', 'Caluroso', 'Extremo']
    )

    # Necesidad de riego
    df['necesidad_riego'] = pd.cut(
        df['precipitacion_mm'],
        bins=[-np.inf, 5, 15, 30, np.inf],
        labels=['Crítico', 'Alto', 'Moderado', 'Bajo']
    )

    # Índice de productividad (combinado)
    df['productividad'] = (
        df['salud_cultivo'] * 0.5 +
        (100 - abs(df['temperatura_celsius'] - 20) * 2).clip(0, 100) * 0.3 +
        df['precipitacion_mm'].clip(0, 50) * 0.4
    ).clip(0, 100)

    # Estación del año (hemisferio sur)
    df['mes'] = df['fecha'].dt.month
    estaciones = {
        12: 'Verano', 1: 'Verano', 2: 'Verano',
        3: 'Otoño', 4: 'Otoño', 5: 'Otoño',
        6: 'Invierno', 7: 'Invierno', 8: 'Invierno',
        9: 'Primavera', 10: 'Primavera', 11: 'Primavera'
    }
    df['estacion'] = df['mes'].map(estaciones)

    return df

datos_juego = calcular_metricas_juego(datos_completos)

# ============================================
# PASO 8: EXPORTAR DATOS
# ============================================

# CSV para análisis
datos_juego.to_csv('cusco_datos_agricolas_juego.csv', index=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.csv")

# JSON para Unity/Godot/Unreal
datos_json = datos_juego.to_dict(orient='records')
import json
with open('cusco_datos_agricolas_juego.json', 'w', encoding='utf-8') as f:
    json.dump(datos_json, f, indent=2, default=str, ensure_ascii=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.json")

# ============================================
# PASO 9: VISUALIZACIÓN
# ============================================

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# NDVI
axes[0].plot(datos_juego['fecha'], datos_juego['NDVI'], color='green', linewidth=2)
axes[0].set_title('NDVI - Salud de la Vegetación en Cusco', fontsize=14, fontweight='bold')
axes[0].set_ylabel('NDVI')
axes[0].grid(True, alpha=0.3)

# Temperatura
axes[1].plot(datos_juego['fecha'], datos_juego['temperatura_celsius'], color='red', linewidth=2)
axes[1].set_title('Temperatura Superficial (°C)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('°C')
axes[1].grid(True, alpha=0.3)

# Precipitación
axes[2].bar(datos_juego['fecha'], datos_juego['precipitacion_mm'], color='blue', alpha=0.6)
axes[2].set_title('Precipitación Diaria (mm)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('mm')
axes[2].grid(True, alpha=0.3)

# Productividad (métrica del juego)
axes[3].plot(datos_juego['fecha'], datos_juego['productividad'], color='purple', linewidth=2)
axes[3].set_title('Índice de Productividad (Métrica del Juego)', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Índice (0-100)')
axes[3].set_xlabel('Fecha')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('cusco_analisis_agricola.png', dpi=300, bbox_inches='tight')
print("📊 Gráfico guardado en: cusco_analisis_agricola.png")
plt.show()

# ============================================
# PASO 10: ESTADÍSTICAS PARA DOCUMENTACIÓN
# ============================================

print("\n" + "="*60)
print("📈 ESTADÍSTICAS PARA TU HACKATHON")
print("="*60)

print(f"\n📅 Período analizado: {datos_juego['fecha'].min()} a {datos_juego['fecha'].max()}")
print(f"📊 Total de observaciones: {len(datos_juego)}")

print("\n🌱 NDVI (Salud de Cultivos):")
print(f"   • Promedio: {datos_juego['NDVI'].mean():.3f}")
print(f"   • Máximo: {datos_juego['NDVI'].max():.3f}")
print(f"   • Mínimo: {datos_juego['NDVI'].min():.3f}")

print("\n🌡️ Temperatura:")
print(f"   • Promedio: {datos_juego['temperatura_celsius'].mean():.1f}°C")
print(f"   • Máxima: {datos_juego['temperatura_celsius'].max():.1f}°C")
print(f"   • Mínima: {datos_juego['temperatura_celsius'].min():.1f}°C")

print("\n💧 Precipitación:")
print(f"   • Total anual: {datos_juego['precipitacion_mm'].sum():.1f} mm")
print(f"   • Promedio diario: {datos_juego['precipitacion_mm'].mean():.1f} mm")
print(f"   • Día más lluvioso: {datos_juego['precipitacion_mm'].max():.1f} mm")

print("\n🎮 Métricas del Juego:")
print(f"   • Productividad promedio: {datos_juego['productividad'].mean():.1f}/100")
print(f"   • Días con estrés extremo: {(datos_juego['estres_calor'] == 'Extremo').sum()}")
print(f"   • Días con riego crítico: {(datos_juego['necesidad_riego'] == 'Crítico').sum()}")

print("\n✅ ¡Datos listos para tu videojuego!")
print("="*60)

In [1]:
# Clonar el repo desde GitHub (solo la primera vez en cada sesión de Colab)
!git clone https://github.com/iAmMazapan/FarmGuardians.git

# Instalar las dependencias desde requirements.txt
!pip install -r FarmGuardians/DataIntegrator/requirements.txt


Cloning into 'FarmGuardians'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 2), reused 19 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 11.87 KiB | 5.93 MiB/s, done.
Resolving deltas: 100% (2/2), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.3 MB/s eta 0:00:00


# Análisis de NDVI y Temperatura en Cusco, Perú

Este notebook permite extraer y analizar datos satelitales de vegetación (NDVI) y temperatura para la región de Cusco, Perú, usando Google Earth Engine. Cada sección incluye explicación y código paso a paso.



# Paso 1: Montar Google Drive

En este paso montaremos Google Drive en Colab. Esto permite:

- Guardar archivos generados, como CSV con datos de NDVI y temperatura.
- Acceder a archivos que tengas en tu Drive, por ejemplo GeoJSON con áreas de estudio.
- Mantener los archivos disponibles mientras trabajamos en el notebook.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Paso 2: Instalar y cargar librerías necesarias

Vamos a usar varias librerías:

- `ee` y `geemap` → para trabajar con Google Earth Engine.
- `pandas` y `numpy` → para manipular datos.
- `matplotlib` → para graficar NDVI y temperatura.
- `scipy.signal.savgol_filter` → para suavizar series de NDVI si hay ruido.

Primero instalamos las librerías necesarias y luego las importamos en Python.


In [ ]:
# Instalar las librerías desde el requirements.txt local
!pip install -r FarmGuardians/DataIntegrator/requirements.txt

# Importar las librerías
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'FarmGuardians/DataIntegrator/requirements.txt'
